In [ ]:
import pyspark
from pyspark.sql import SparkSession
app_name = "accidentes"
master = "local[*]"
spark = (SparkSession.builder
    .master(master)
    .config("spark.driver.cores", 1)
    .appName(app_name)
    .getOrCreate() )
sc = spark.sparkContext
print ('SparkContext created')

In [ ]:
import urllib.request
def getLinesFromUrl(url:str):
    """
    Downloads content from a url and
    creates a text based RDD
    """
    response = urllib.request.urlopen(url)
    data = response.read().decode('utf-8')
    lines = data.split('\n')
    # creates a RDD for the book
    return sc.parallelize(lines)


In [ ]:
huckleberry_lines = getLinesFromUrl ('https://www.gutenberg.org/files/76/76-0.txt')
hamlet_lines = getLinesFromUrl ('https://www.gutenberg.org/files/2265/2265.txt')

In [ ]:
import re

def getRDDWords(book):
    def clean_line(line):
        """
        Remove \ufeff\r characters
        Remove \t \n \r
        Remove additional characters
        """
        return line.replace('\ufeff\r', '').\
            replace('\t', ' ').replace('\n', '').replace('\r', '').\
            replace('(', '').replace(')', '').replace("'", '').\
            replace('"', '').replace(',', ''). replace('.', '').\
            replace('*', '')

    def normalize_tokenize(line):
        """
        Normalize: lowercase
        tokenize: split in tokens
        """
        return re.sub('\s+', ' ', line).strip().lower().split(' ')

    return book.map(lambda x: clean_line(x)).\
        filter(lambda x: x != '').flatMap(normalize_tokenize) #.\
        #filter(lambda x: len(x)>3).\
        #map (lambda x: (x, 1)).\
        #reduceByKey (lambda x, y: x + y)


In [ ]:
huckleberry_words = getRDDWords (huckleberry_lines)
huckleberry_counts = huckleberry_words.map (lambda x: (x, 1)).\
                                       reduceByKey (lambda x, y: x + y)
print ('Tokens: {} / First {}'.format (huckleberry_counts.count(), huckleberry_counts.take(5)))

In [ ]:
hamlet_words = getRDDWords (hamlet_lines)
hamlet_counts = hamlet_words.map (lambda x: (x, 1)).\
                                       reduceByKey (lambda x, y: x + y)
print ('Tokens: {} / First {}'.format (hamlet_counts.count(), hamlet_counts.take(5)))

# Calcular la media de la longitud de palabra
---
<br>
puede hacer uso de la funcion [RDD].reduce(f)
<br>
<br>
<br>
<br>
<br>
<br>

# Desviación estándar
---
$\Large\text{DE} = \sqrt{\dfrac{\sum\limits_{}^{}{{\lvert x-\mu\rvert^2}}}{N}}$

### donde: 
* $\text{x}$ son los elementos de la población, cada una de las longitudes
* $\mu$ es la media de la población, en este caso la longitud media de las palabras
* $\text{N}$ es el número de elementos de la población, la cantidad de palabras
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

# 5 palabras más frecuentes
---
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

In [ ]:
#Elimina el contexto spark
#Debe ser la última sentencia
sc.stop()